In [1]:
import ipy_table
from collections import Counter
%run ../aux/postgres.py

import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.util import ngrams

import warnings
warnings.filterwarnings('ignore')

tknzr = TweetTokenizer()
#%run functions/preproc.py
#%run functions/classificacao.py

postgres functions loaded...


In [2]:
def tokensCleaner(txt):
    tokens = tknzr.tokenize(txt)
    stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
    clean_tokens = []
    for token in tokens:
        if token[:1] == '#':
            token = token[1:]
        if not token.isalpha():
            continue
        if token in stop:
            continue
        if token[:4] == 'http':
            continue
        if token[:1] == '@':
            continue
        if len(token) <= 3:
            continue

        clean_tokens.append(token)
    return clean_tokens

In [3]:
cmd = ("select cl.codclassificacao, cl.descricao, count(codtweet) "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "where codresultadoanaliseprojeto = 20 "
       "GROUP BY 1 , 2"
      )
res = query(cmd)
res

[(227, 'personal', 34535),
 (226, 'service', 154619),
 (228, 'sale', 9379),
 (229, 'clinical', 60886),
 (223, 'information', 18290),
 (224, 'product', 22278)]

In [4]:
for r in res:
    print(r)
    print(r[1])
    # loading from database
    cmd = ("select t.text, cl.codclassificacao "
           "from resultadoanaliseprojetotweet r "
           "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
           "inner join tweet t on r.codtweet = t.codtweet "
           "where codresultadoanaliseprojeto = 20 and cl.codclassificacao = " + str(r[0]))
    res01 = query(cmd)    
    print(cmd)
    print(len(res01))
    
    # concatenando texto
    txt = ''
    for row in res01:
        txt = txt + ' ' + row[0].lower()    
    
    # pré processamento
    clean_tokens = tokensCleaner(txt)
    
    # unigramas
    unigramas = ngrams(clean_tokens,1)
    unigramas_count = Counter(unigramas).most_common()
    arquivo = "data/unigramas_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "w") as f:
        for b in unigramas_count:
            f.write(b[0][0]+";"+str(b[1])+"\n")    
    
    # bigramas
    bigramas = ngrams(clean_tokens, 2)
    bigramas_count = Counter(bigramas).most_common()
    arquivo = "data/bigramas_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "w") as f:
        for b in bigramas_count:
            f.write(b[0][0]+";"+b[0][1]+";"+str(b[1])+"\n")    

    # trigramas
    trigramas = ngrams(clean_tokens, 3)
    trigramas_count = Counter(trigramas).most_common()
    arquivo = "data/trigramas_" + str(r[1]) + ".csv"
    print(arquivo)
    with open(arquivo, "w") as f:
        for b in trigramas_count:
            f.write(b[0][0]+";"+b[0][1]+";"+b[0][2]+";"+str(b[1])+"\n")    
        
    print()
    

(227, 'personal', 34535)
personal
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 20 and cl.codclassificacao = 227
34535
data/unigramas_personal.csv
data/bigramas_personal.csv
data/trigramas_personal.csv

(226, 'service', 154619)
service
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanaliseprojeto = 20 and cl.codclassificacao = 226
154619
data/unigramas_service.csv
data/bigramas_service.csv
data/trigramas_service.csv

(228, 'sale', 9379)
sale
select t.text, cl.codclassificacao from resultadoanaliseprojetotweet r inner join classificacao cl on cl.codclassificacao = r.codclassificacao inner join tweet t on r.codtweet = t.codtweet where codresultadoanalisepr

In [5]:
clean_tokens = tokensCleaner(txt)
print(clean_tokens[:10])

['coxo', 'contra', 'angle', 'handpiece', 'reduction', 'check', 'torque', 'wrench', 'nobel', 'biocare']


In [6]:
clean_tokens = tokensCleaner(txt)

In [7]:
# unigramas
ungs = ngrams(clean_tokens,1)
aaa = Counter(ungs).most_common()
with open("data/unigramas.csv", "w") as f:
    for b in aaa:
        print("%s;%s" % (b[0][0], b[1] ))
        f.write(b[0][0]+";"+str(b[1])+"\n")

cost;4137
bone;1898
torque;1587
wrench;1556
check;1507
know;1321
straumann;1206
business;985
adin;976
announce;974
chicago;966
success;963
social;958
technology;956
reuters;956
alsgd;956
newyork;956
nytimes;954
bloomberg;954
thestreet;954
jimmyfallon;954
forbes;954
nasdaq;954
digitalmarkiting;954
ihub;954
networking;954
foxnews;954
nobel;941
spineguard;936
titanium;934
system;933
handpiece;869
teeth;835
restoration;812
rate;812
reduction;794
biocare;764
angle;704
contra;678
motor;664
surgical;652
drivers;640
graft;588
placement;577
tooth;571
systems;556
dentalimplants;543
short;527
screw;526
long;516
made;488
study;479
internal;456
risk;447
dentistry;439
types;433
history;424
broken;398
ceramic;396
successful;396
need;385
trust;381
mouth;379
failure;379
removal;374
help;366
people;359
surgery;356
remover;354
abutment;353
retrieval;343
reservoir;342
overall;339
universal;328
brief;328
reduces;324
full;321
brushless;321
used;317
driver;315
approved;308
invisalign;299
designed;298
antibac

mechanical;13
temporary;13
allows;13
lower;13
feels;13
failed;13
devices;13
lifetime;13
reach;13
theyre;13
renishaw;13
collaborated;13
innovator;13
covers;13
contact;13
became;13
individuals;13
estimates;13
patented;13
ideal;12
drills;12
software;12
auto;12
smart;12
kensington;12
introduction;12
thank;12
announced;12
modern;12
inner;12
osteotome;12
osseo;12
aaid;12
switch;12
celebrity;12
events;12
videos;12
steel;12
cavity;12
conference;12
effect;12
week;12
multiple;12
discover;12
sign;12
performance;12
teething;12
troubles;12
project;12
grants;12
manufacturing;12
cover;12
foods;12
pain;12
safety;12
micro;12
biofilms;12
weird;12
retweeted;12
indilens;12
sinuslift;12
desirable;12
weeks;12
still;12
expect;12
barthmanndentureclinic;12
denturist;12
guys;12
environment;12
goldadapt;12
engaging;12
charlottedentistry;12
grows;12
collar;12
neobiotech;12
falseteeth;12
meoplant;12
cellulose;12
plac;12
length;11
mixed;11
presentations;11
irradiated;11
block;11
sample;11
prices;11
removable;11
ort

torrado;4
denta;4
cultivable;4
polymicrobial;4
hollywood;4
maintenance;4
dioxide;4
colgate;4
cynamed;4
nature;4
lose;4
nowadays;4
comparing;4
endosseous;4
harvesting;4
seen;4
commercials;4
discovery;4
riding;4
yorkshire;4
reznick;4
speaks;4
blueprint;4
worldsummittour;4
edge;4
dentalimplantsindia;4
cadaver;4
coding;4
billing;4
towards;4
michelle;4
interactions;4
perfectly;4
diagnosis;4
schein;4
isles;4
scilly;4
severe;4
faqs;4
adjustable;4
stainless;4
outs;4
opened;4
osstell;4
detect;4
enables;4
missingtooth;4
hygiene;4
longevity;4
differences;4
compatibility;4
called;4
prototypes;4
produced;4
idea;4
located;4
reason;4
integrates;4
jaws;4
smallest;4
seamlessly;4
csailor;4
knowledge;4
losaltosdentist;4
sanjose;4
bayareadentist;4
bayarea;4
losaltos;4
lascrucesnm;4
lascruces;4
simplifying;4
osteotomy;4
mile;4
safer;4
prior;4
producing;4
augmented;4
practitioners;4
classification;4
ought;4
forteimplantcenter;4
comfortable;4
tampa;4
cylinders;4
starting;4
package;4
room;4
loose;4
outcomes;4

retaining;2
silicone;2
cleft;2
cleftlip;2
palate;2
trial;2
crazy;2
automatic;2
skills;2
achieves;2
milestone;2
minority;2
fails;2
shell;2
vitallium;2
functional;2
botched;2
intrinsic;2
chief;2
dentofacial;2
deformity;2
workshops;2
planting;2
intrusion;2
arches;2
diss;2
nearby;2
saying;2
terrifying;2
studying;2
blend;2
align;2
wondered;2
directory;2
moment;2
spend;2
recipients;2
pretty;2
evaluate;2
unfortunately;2
ukengnews;2
accidents;2
digitaldentistry;2
hopefully;2
allograft;2
april;2
dentalce;2
bionika;2
excess;2
reflects;2
finest;2
rutgers;2
resolution;2
israel;2
confident;2
nine;2
reports;2
extractions;2
dentalimplantrestoration;2
emergence;2
significantly;2
written;2
pubmed;2
pinch;2
demonstrate;2
mfghouruk;2
accelerate;2
cared;2
scaler;2
curettes;2
sisc;2
segment;2
dominated;2
maxillofacialsurgery;2
partha;2
miniaturized;2
recently;2
playing;2
underlying;2
bonded;2
suitable;2
iowadentist;2
retrieve;2
connects;2
cigarette;2
complimentary;2
yeah;2
sector;2
regrown;2
endodontist;2


viruses;1
healthtech;1
fights;1
dncforum;1
keepkatierich;1
coco;1
apart;1
trumpcare;1
obamacare;1
maga;1
privacy;1
myhillsdentist;1
lifecore;1
medication;1
kathleen;1
collagene;1
uniqueness;1
assessed;1
forensic;1
bitemark;1
delivers;1
lifehack;1
subscribe;1
reductioncontra;1
upset;1
anemia;1
rotational;1
castable;1
burn;1
brushles;1
platinum;1
excited;1
xnkl;1
whatsstoppingyou;1
foramina;1
holdthefloor;1
softenabadguy;1
janethevirgin;1
ifsgreenbudget;1
girlsinstem;1
sajid;1
javid;1
mitchellajosephsdds;1
tenting;1
harmony;1
preethanath;1
edent;1
sidcup;1
nationaltoothacheday;1
scripsnow;1
enjoy;1
clarify;1
released;1
chemicals;1
fought;1
imitate;1
hypoplastic;1
fixing;1
interested;1
originated;1
howard;1
farran;1
fridayfeeling;1
matt;1
reeves;1
lucifer;1
lust;1
goagfstu;1
colonelsdentalspa;1
trea;1
strategi;1
implantable;1
bioprinted;1
purest;1
nations;1
wellbeing;1
feeling;1
newdentist;1
secondthoughts;1
southkorea;1
bottomup;1
equities;1
revivedwithdentalimplant;1
mino;1
esearch;1
kn

trading;1
newer;1
counts;1
endopore;1
nobody;1
branemårk;1
knew;1
picdental;1
minute;1
capture;1
photogrammetry;1
estheticdentistry;1
dentalstudent;1
補綴専門医;1
インプラント;1
審美歯科;1
turnout;1
ratoath;1
night;1
glenn;1
mcevoy;1
andrea;1
referring;1
dentalcheckup;1
typesofimplants;1
stockmarket;1
unsightly;1
gaps;1
glasscomposites;1
ceramicglasses;1
nanocomposites;1
graphenenanotechnology;1
dentalceramics;1
accepts;1
disadvantage;1
baseball;1
closer;1
dentalhygienist;1
diagnolounge;1
diagno;1
arguably;1
smilebetter;1
mondaymood;1
brussels;1
pills;1
cosmetics;1
discussions;1
expo;1
haines;1
glacier;1
lodge;1
weinstein;1
melnick;1
douglas;1
margo;1
woll;1
generous;1
replaceable;1
friscotx;1
fancy;1
newsscientists;1
multibeam;1
module;1
gram;1
grams;1
greedy;1
regenerated;1
substantially;1
remodeling;1
aimed;1
investigate;1
lowering;1
somewhat;1
sauhy;1
tired;1
prepping;1
shinhung;1
gadgets;1
foresight;1
breath;1
famework;1
prodeuct;1
hence;1
implantschennai;1
implantprocedure;1
gasser;1
terri;1
pe

In [8]:
# bigramas
ungs = ngrams(clean_tokens, 2)
aaa = Counter(ungs).most_common()
for b in aaa:
    print("%s;%s;%s" % (b[0][0], b[0][1], b[1] ))

cost;cost;2588
torque;wrench;1366
nytimes;reuters;954
reuters;bloomberg;954
bloomberg;thestreet;954
thestreet;jimmyfallon;954
jimmyfallon;forbes;954
forbes;nasdaq;954
nasdaq;chicago;954
chicago;digitalmarkiting;954
digitalmarkiting;ihub;954
ihub;newyork;954
newyork;social;954
social;networking;954
networking;business;954
business;foxnews;954
spineguard;adin;933
foxnews;alsgd;821
nobel;biocare;759
success;rate;730
contra;angle;665
bone;graft;534
check;torque;525
alsgd;announce;495
placement;nytimes;495
adin;systems;475
systems;announce;468
alsgd;spineguard;461
announce;spineguard;461
adin;nytimes;459
technology;placement;456
announce;technology;454
broken;screw;387
angle;handpiece;374
technology;trust;368
short;long;354
screw;removal;335
overall;success;331
trust;technology;328
reduction;contra;326
removal;remover;307
remover;retrieval;307
retrieval;restoration;307
reservoir;reduces;298
brief;history;295
press;ceramic;252
check;straumann;251
hydrualic;press;251
bone;grafting;248
austral

immediate;early;13
early;conventional;13
tool;straumann;12
membrane;hold;12
press;release;12
internal;people;12
compatible;healing;12
abutment;bottle;12
bottle;shape;12
grafting;check;12
crown;kensington;12
oral;maxillofacial;12
healthy;health;12
demonstration;model;12
lift;bone;12
adapter;osseo;12
reduction;surgical;12
dentistry;tech;12
surgery;brushless;12
navigation;system;12
titanium;screw;12
handpiece;cordless;12
cordless;endo;12
rate;dentalimplant;12
allow;teething;12
teething;troubles;12
grafting;material;12
straumann;cost;12
bone;mill;12
guide;check;12
fight;infections;12
reservoir;reduce;12
sirona;lead;12
oral;health;12
system;procedures;12
sale;motor;12
treatmentoftheday;dentalimplants;12
successful;lfnin;12
zirconia;titanium;12
years;become;12
become;desirable;12
desirable;alternative;12
alternative;methods;12
methods;replacing;12
health;barthmanndentureclinic;12
barthmanndentureclinic;denturist;12
straumann;slactive;12
cost;success;12
cost;trending;12
traditional;cost;12
su

angle;suitcase;5
help;check;5
maui;posted;5
surgical;torque;5
know;process;5
ratio;hand;5
three;torques;5
mhealth;hcit;5
hcit;hcsm;5
hcsm;meded;5
people;need;5
impression;high;5
many;people;5
motor;case;5
year;waiting;5
tools;bone;5
know;patients;5
implantology;nobel;5
biocare;systems;5
systems;oral;5
rehabilitation;share;5
handpiece;tool;5
lost;bone;5
handpiece;titanium;5
time;make;5
advantages;nobel;5
titanium;know;5
complete;guide;5
healing;preparation;5
association;oral;5
maxillofacial;surgeons;5
healing;caps;5
ratchet;straumann;5
platform;alpha;5
model;demonstration;5
slow;contra;5
handpiece;mini;5
dentist;straight;5
bringing;life;5
life;dead;5
dead;tooth;5
advantages;drawbacks;5
check;bego;5
bego;surgery;5
check;alphabio;5
alphabio;surgery;5
check;megagen;5
quality;transfer;5
reduction;latch;5
handpiece;surgery;5
benefits;high;5
system;handpiece;5
higher;success;5
titanium;sleeve;5
crown;titanium;5
straight;narrow;5
biocare;nobel;5
titanium;ball;5
composition;titanium;5
screw;rou

long;ratchet;3
short;ratchet;3
internal;post;3
integration;torque;3
impact;success;3
conventional;care;3
care;advise;3
wrench;titanium;3
failure;nobel;3
sealed;made;3
racine;wisconsin;3
shape;straumann;3
motor;push;3
unit;plastic;3
plastic;titanium;3
sleeve;internal;3
patient;atlantissolutions;3
heard;exactly;3
cosmeticdentistry;know;3
template;types;3
torques;handpiece;3
parts;straight;3
mete;system;3
really;worth;3
people;considering;3
ratio;reduction;3
graft;templates;3
graft;slides;3
bone;demonstration;3
brushless;swiss;3
process;straight;3
handpiece;made;3
ceramic;study;3
bravo;system;3
baistra;motor;3
manchest;randalvstone;3
system;suitcase;3
handpiece;technology;3
common;questions;3
presentations;types;3
style;sgdx;3
serve;types;3
handpiece;braces;3
impression;powerpoint;3
integration;investigators;3
investigators;found;3
type;instrument;3
ever;wonder;3
used;type;3
type;metal;3
shuttles;rockets;3
surgeon;pioneers;3
pioneers;system;3
system;daniel;3
vadodara;traditionally;3
tradi

post;surgical;2
system;extensively;2
extensively;scientifically;2
scientifically;documented;2
parts;consist;2
consist;actual;2
actual;fixture;2
fixture;abutment;2
artistry;answers;2
common;related;2
related;queries;2
queries;dentist;2
dentist;cosmeticdentistry;2
cosmeticdentistry;orals;2
remover;straumann;2
check;full;2
units;titanium;2
internal;french;2
handpiece;ceramic;2
includes;titanium;2
titanium;artificial;2
total;rehabilitation;2
rehabilitation;oral;2
oral;cavity;2
cavity;bone;2
grafting;conference;2
conference;turkey;2
turkey;konya;2
history;patients;2
know;sale;2
sale;machine;2
handpiece;tube;2
tube;brief;2
permadontics;learning;2
adin;spiral;2
method;know;2
highly;successful;2
replace;dentsply;2
dentsply;tool;2
information;bone;2
nobelbio;sybron;2
sybron;dentium;2
dentium;tool;2
wrench;impact;2
titani;impact;2
obstruction;would;2
track;record;2
biomet;straumann;2
handpiece;main;2
rescue;surgical;2
complete;straumann;2
equipment;surgical;2
square;direct;2
sizes;broken;2
metho

...

procedures;printing;1
guides;refer;1
dentist;printing;1
guides;system;1
cylindrical;rigid;1
rigid;plastic;1
plastic;ring;1
ring;system;1
cylindrical;flapless;1
flapless;surgery;1
piece;screw;1
screw;soft;1
soft;bone;1
technology;makes;1
makes;possible;1
possible;seeing;1
surgeon;four;1
four;continued;1
continued;work;1
work;boland;1
boland;invent;1
invent;innovative;1
innovative;dynamic;1
dynamic;company;1
company;globally;1
globally;active;1
active;manufacturer;1
systems;work;1
work;coming;1
coming;awareness;1
month;little;1
little;favorite;1
favorite;advantages;1
dentalimplants;maintain;1
maintain;healthy;1
healthy;need;1
need;simple;1
simple;cleaning;1
cleaning;sterling;1
sterling;considering;1
wrong;considering;1
wrong;causes;1
failure;typically;1
visit;restoration;1
prosthesis;personalized;1
medicine;another;1
another;activity;1
activity;technology;1
technology;already;1
guides;work;1
printing;work;1
work;similarly;1
similarly;teeth;1
teeth;help;1
help;shield;1
shield;maintain;1
m

company;paltop;1
paltop;accelmed;1
accelmed;million;1
million;financing;1
financing;round;1
round;patented;1
patented;protocol;1
protocol;made;1
biocare;hand;1
hand;doctor;1
doctor;paulo;1
paulo;malo;1
malo;consist;1
consist;placing;1
placing;four;1
four;titanium;1
titanium;maxilla;1
maxilla;straight;1
straight;tilted;1
tilted;provides;1
provides;increased;1
increased;anchorage;1
anchorage;stability;1
stability;screwed;1
screwed;titanium;1
titanium;teeth;1
plus;simpladent;1
simpladent;implantation;1
simple;swift;1
swift;contrary;1
contrary;traditional;1
traditional;implantation;1
process;simpladent;1
simpladent;immediate;1
loading;implantation;1
implantation;possible;1
possible;restore;1
smile;within;1
within;using;1
using;strategic;1
strategic;tooth;1
plus;failure;1
failure;symptoms;1
symptoms;remedies;1
remedies;atrophic;1
reconstruction;autogenous;1
autogenous;iliac;1
iliac;graft;1
graft;guided;1
guided;artificial;1
artificial;screws;1
screws;info;1
visit;check;1
ratchet;digital;1
p

antibacterial;related;1
related;prosthetic;1
prosthetic;components;1
components;could;1
infection;prevent;1
prevent;oral;1
surgeon;ongoing;1
ongoing;care;1
boland;number;1
number;full;1
arch;prosthesis;1
survival;complication;1
complication;rates;1
rates;miss;1
survival;cost;1
cost;product;1
product;alert;1
alert;whip;1
whip;introduces;1
introduces;veriguide;1
veriguide;class;1
class;certified;1
certified;resin;1
resin;material;1
material;developed;1
developed;production;1
production;high;1
high;precision;1
precision;surgical;1
guides;surgery;1
surgery;learn;1
learn;purchase;1
purchase;post;1
know;number;1
number;influence;1
influence;survival;1
rate;prosthesis;1
survival;posted;1
reddit;miss;1
survival;fact;1
funfacts;houstontx;1
houstontx;labradors;1
mail;powerful;1
powerful;digital;1
motor;traditional;1
falseteeth;fortworthtx;1
fortworthtx;traditional;1
falseteeth;garlandtx;1
garlandtx;traditional;1
falseteeth;atlantaga;1
atlantaga;mariettaga;1
mariettaga;post;1
blog;animal;1
animal

found;dentzz;1
dentzz;hear;1
hear;journey;1
journey;dentzz;1
dentzz;dentzz;1
dentzz;dentaltourism;1
dentaltourism;topdentalclinic;1
topdentalclinic;smilemakeover;1
smilemakeover;dentalimplants;1
dentalimplants;post;1
blog;review;1
review;study;1
compares;machined;1
machined;sandblasted;1
sandblasted;surfaces;1
surfaces;high;1
rate;uncommon;1
uncommon;fall;1
fall;still;1
still;fail;1
fail;learn;1
learn;causes;1
causes;dentalimplants;1
dentalimplants;fall;1
fall;post;1
infections;evidence;1
success;limited;1
limited;success;1
aaoms;thankful;1
thankful;gotten;1
gotten;know;1
know;cathy;1
cathy;bone;1
patients;nice;1
nice;give;1
give;positive;1
positive;feedback;1
feedback;experience;1
experience;patientstories;1
patientstories;people;1
people;metal;1
detectors;limited;1
success;nasa;1
missiles;dentalimplants;1
dentalimplants;northshoreoralsurgeryllc;1
northshoreoralsurgeryllc;photobiomodulation;1
dentalnews;photobiomodulation;1
success;replace;1
root;create;1
create;stability;1
stability;

veneers;zircon;1
zircon;porcelain;1
porcelain;bone;1
graft;aesthetics;1
aesthetics;everything;1
everything;treatment;1
treatment;hotel;1
hotel;accommodation;1
accommodation;airport;1
airport;transfers;1
transfers;celebrity;1
celebrity;brand;1
matters;prognostic;1
prognostic;model;1
model;outcome;1
outcome;nobel;1
biocare;disease;1
disease;year;1
cost;bank;1
bank;costs;1
tooth;bank;1
bank;different;1
know;brand;1
matters;post;1
post;impressive;1
rate;make;1
time;titanium;1
posts;lovesmile;1
lovesmile;tooth;1
replacement;closest;1
closest;nature;1
nature;possible;1
possible;best;1
best;lovesmile;1
lovesmile;practice;1
practice;titanium;1
posts;small;1
post;custom;1
custom;made;1
made;false;1
false;tooth;1
tooth;fixed;1
fixed;find;1
find;many;1
many;othe;1
othe;performed;1
learn;ridge;1
along;make;1
restoration;closest;1
posts;free;1
around;email;1
email;dentalschool;1
dentalschool;dentistry;1
dentistry;dentista;1
dentista;dentalcrowns;1
dentalcrowns;endodontics;1
endodontics;nobelbiocare

In [9]:
# trigramas
ungs = ngrams(clean_tokens, 3)
aaa = Counter(ungs).most_common()
for b in aaa:
    print("%s;%s;%s;%s" % (b[0][0], b[0][1], b[0][2], b[1]))

cost;cost;cost;1871
nytimes;reuters;bloomberg;954
reuters;bloomberg;thestreet;954
bloomberg;thestreet;jimmyfallon;954
thestreet;jimmyfallon;forbes;954
jimmyfallon;forbes;nasdaq;954
forbes;nasdaq;chicago;954
nasdaq;chicago;digitalmarkiting;954
chicago;digitalmarkiting;ihub;954
digitalmarkiting;ihub;newyork;954
ihub;newyork;social;954
newyork;social;networking;954
social;networking;business;954
networking;business;foxnews;954
business;foxnews;alsgd;821
placement;nytimes;reuters;495
check;torque;wrench;493
spineguard;adin;systems;469
adin;systems;announce;468
alsgd;spineguard;adin;461
systems;announce;spineguard;461
announce;spineguard;adin;461
spineguard;adin;nytimes;459
adin;nytimes;reuters;459
alsgd;announce;technology;454
announce;technology;placement;454
technology;placement;nytimes;454
foxnews;alsgd;announce;412
foxnews;alsgd;spineguard;409
contra;angle;handpiece;367
overall;success;rate;329
technology;trust;technology;328
trust;technology;trust;328
broken;screw;removal;327
reductio

solid;approved;piece;7
osteotome;concave;offset;7
upto;preset;premium;7
success;rate;brief;7
rate;brief;history;7
drivers;hexa;type;7
hexa;type;straumann;7
long;short;straumann;7
short;torque;drivers;7
straumann;check;bone;7
graft;sinus;surgery;7
health;healthy;mhealth;7
healthy;mhealth;digitalhealth;7
sealed;camlog;promote;7
camlog;promote;plus;7
disposable;tube;tooth;7
tube;tooth;irrigation;7
wrench;ratchet;wrench;7
would;cause;failure;7
machine;brushless;motor;7
bruxism;failures;multilevel;7
failures;multilevel;mixed;7
multilevel;mixed;effects;7
mixed;effects;parametric;7
effects;parametric;survival;7
parametric;survival;analysis;7
survival;analysis;approach;7
want;know;want;7
know;want;know;7
coxo;surgery;brushless;7
artificial;tooth;root;7
nobel;biocare;square;7
teeth;mouth;grow;7
wide;enough;support;7
enough;support;bone;7
graft;material;added;7
material;added;small;7
added;small;ridge;7
small;ridge;space;7
surgical;motor;system;7
brushless;micromotor;reduction;7
reported;success

mile;maui;dentists;4
maui;dentists;supply;4
cost;know;made;4
titanium;metal;biocompatible;4
metal;biocompatible;bone;4
biocompatible;bone;structure;4
bone;structure;replace;4
structure;replace;missing;4
missing;teeth;keep;4
teeth;keep;mouth;4
keep;mouth;happy;4
mouth;happy;healthy;4
designed;provide;safer;4
cost;cost;fact;4
read;help;know;4
help;know;overall;4
straight;abutment;conical;4
cost;material;used;4
body;life;cost;4
advantages;improved;appearance;4
improved;speech;improved;4
speech;improved;comfort;4
improved;comfort;easier;4
comfort;easier;eating;4
improved;oral;health;4
learning;cost;cost;4
making;room;bone;4
room;bone;augmentation;4
cost;system;specifically;4
healthcare;engineering;cost;4
want;know;infographic;4
could;cost;fence;4
metal;titanium;rockets;4
titanium;rockets;something;4
could;alsgd;announce;4
article;explains;alsgd;4
artificial;roots;inserted;4
detector;system;locate;4
system;locate;submerged;4
locate;submerged;various;4
submerged;various;brands;4
various;bran

biocare;straumann;totalimplant;2
adapter;megagen;totalimplant;2
etiii;fixture;regular;2
fixture;regular;premounted;2
regular;premounted;nobel;2
perform;full;range;2
full;range;bone;2
range;bone;grafting;2
straumann;check;expander;2
membrane;hold;people;2
hold;people;replacing;2
rate;check;expander;2
check;expander;wrench;2
expander;wrench;augmentation;2
wrench;augmentation;bone;2
bone;expander;complete;2
expander;complete;surgical;2
complete;surgical;wrench;2
surgical;wrench;square;2
sinus;surgery;periodontic;2
surgery;periodontic;demonstration;2
periodontic;demonstration;teeth;2
phones;metal;cell;2
metal;cell;phones;2
phones;metal;people;2
metal;people;replacing;2
zygomatic;bone;evidence;2
bone;tack;bone;2
totalimplantcom;check;expander;2
imaging;powerpoint;slide;2
powerpoint;powerpoint;presentation;2
wrench;torque;drivers;2
torque;drivers;nobel;2
drivers;nobel;astra;2
nobel;astra;premium;2
astra;premium;collection;2
surgical;titanium;rods;2
titanium;rods;mill;2
laminaâ;straight;abutm

bone;growth;million;2
growth;million;people;2
years;percent;brief;2
percent;brief;history;2
brief;history;know;2
know;similarities;differences;2
similarities;differences;mini;2
mini;traditional;offer;2
brief;history;nobel;2
snappy;abutment;broken;2
abutment;broken;screw;2
jeremy;could;soon;2
could;soon;become;2
soon;become;history;2
become;history;stem;2
stem;cell;banking;2
cell;banking;news;2
banking;news;reviews;2
news;reviews;health;2
reviews;health;teeth;2
titanium;impression;coping;2
impression;coping;abutment;2
coping;abutment;nobel;2
avoid;basal;nobel;2
basal;nobel;biocare;2
full;mouth;braces;2
mouth;braces;arundati;2
braces;arundati;krishnaraj;2
retrieval;restoration;help;2
restoration;help;preserve;2
sept;engineering;system;2
engineering;system;daniel;2
system;daniel;rittel;2
daniel;rittel;organiza;2
restoration;check;drivers;2
replace;abutment;nobel;2
various;types;facts;2
types;facts;behind;2
success;rate;surgery;2
abutment;know;overall;2
tray;plastic;nobel;2
plastic;nobel;b

...

labradors;innovative;approach;1
innovative;approach;replacing;1
approach;replacing;missing;1
missing;teeth;estimated;1
teeth;estimated;million;1
estimated;million;people;1
million;people;visit;1
people;visit;link;1
visit;link;breaking;1
link;breaking;news;1
euthanisia;labradors;composition;1
labradors;composition;titanium;1
vanadium;alloy;teeth;1
alloy;teeth;tomorrow;1
teethtomorrow;astoria;look;1
astoria;look;difference;1
dentalimplants;harrow;american;1
harrow;american;academy;1
dentalimplants;cosmeticdentistry;columbusoh;1
cosmeticdentistry;columbusoh;american;1
columbusoh;american;academy;1
dentalimplants;cosmeticdentistry;clemmonsnc;1
cosmeticdentistry;clemmonsnc;fact;1
clemmonsnc;fact;type;1
cosmeticdentistry;longbeachca;american;1
longbeachca;american;academy;1
cosmeticdentistry;garlandtx;generally;1
cosmeticdentistry;glendaleaz;find;1
glendaleaz;find;ceramics;1
find;ceramics;updates;1
ceramics;updates;materials;1
stronger;cheaper;graphene;1
cheaper;graphene;carbonnanotubes;1
gr

magazine;cyanide;pill;1
cyanide;pill;bone;1
pill;bone;grafting;1
grafting;process;process;1
process;process;process;1
process;process;technology;1
process;technology;rebuild;1
smilemakeover;watch;look;1
watch;look;feel;1
since;economics;fractured;1
economics;fractured;screw;1
screw;retrieval;good;1
retrieval;good;explanation;1
good;explanation;kind;1
explanation;kind;complications;1
implantdentistry;sandiegoca;brief;1
sandiegoca;brief;history;1
replacement;innovation;replace;1
innovation;replace;multiple;1
replace;multiple;teeth;1
teeth;know;require;1
know;require;bone;1
bone;graft;future;1
graft;future;bone;1
future;bone;graft;1
bone;graft;pricing;1
graft;pricing;affordable;1
pricing;affordable;predictable;1
affordable;predictable;code;1
predictable;code;code;1
code;code;depending;1
code;depending;amount;1
depending;amount;bone;1
amount;bone;required;1
bone;required;contact;1
required;contact;quote;1
contact;quote;today;1
quote;today;bone;1
today;bone;grafting;1
bone;grafting;success;

cyst;without;bone;1
grafting;material;foll;1
material;foll;regular;1
foll;regular;platform;1
connection;titanium;regular;1
titanium;regular;platform;1
connection;titanium;supply;1
titanium;supply;astm;1
supply;astm;grade;1
astm;grade;bone;1
grade;bone;titanium;1
bone;titanium;forging;1
titanium;forging;surgical;1
forging;surgical;medical;1
surgical;medical;alloy;1
medical;alloy;grade;1
alloy;grade;rods;1
grade;rods;slim;1
rods;slim;ball;1
slim;ball;attachment;1
abutment;internal;post;1
internal;post;orthocell;1
post;orthocell;completes;1
orthocell;completes;celgro;1
completes;celgro;study;1
celgro;study;accelerating;1
study;accelerating;treatments;1
accelerating;treatments;spiral;1
treatments;spiral;narrow;1
sterilized;titanium;materials;1
titanium;materials;used;1
titanium;zirconia;impressive;1
zirconia;impressive;success;1
rate;spiral;conical;1
platform;sterilized;game;1
sterilized;game;time;1
game;time;game;1
time;game;implantology;1
game;implantology;system;1
implantology;system;bo

material;usually;made;1
usually;made;first;1
made;first;commercial;1
first;commercial;product;1
commercial;product;implanova;1
product;implanova;boasts;1
implanova;boasts;system;1
boasts;system;designed;1
system;designed;simplify;1
designed;simplify;placement;1
simplify;placement;restoration;1
placement;restoration;well;1
restoration;well;overcome;1
well;overcome;shortcomings;1
overcome;shortcomings;existing;1
shortcomings;existing;systems;1
existing;systems;call;1
systems;call;today;1
call;today;info;1
today;info;know;1
info;know;million;1
million;people;dentalimplants;1
people;dentalimplants;nationaltoothfairyday;1
dentalimplants;nationaltoothfairyday;dentalimplantmonth;1
nationaltoothfairyday;dentalimplantmonth;check;1
dentalimplantmonth;check;video;1
check;video;exactly;1
video;exactly;vmdental;1
exactly;vmdental;patienteducation;1
vmdental;patienteducation;technology;1
patienteducation;technology;rebuild;1
smilemakeover;watch;worms;1
watch;worms;exploiting;1
worms;exploiting;women

sinus;infection;take;1
infection;take;look;1
look;article;pros;1
article;pros;cons;1
pros;cons;overseas;1
cons;overseas;anatomy;1
overseas;anatomy;bleached;1
anatomy;bleached;blonde;1
fuck;somewhere;solicitors;1
somewhere;solicitors;flag;1
solicitors;flag;clothing;1
flag;clothing;roid;1
clothing;roid;ragers;1
roid;ragers;hoverounds;1
ragers;hoverounds;ancient;1
hoverounds;ancient;history;1
ancient;history;costs;1
history;costs;advantages;1
laperiodonticsanddentalimplants;dentures;sedation;1
dentures;sedation;wisdom;1
aaoms;fullscope;different;1
fullscope;different;types;1
lifting;open;join;1
open;join;conference;1
join;conference;explore;1
conference;explore;latest;1
explore;latest;researches;1
latest;researches;field;1
researches;field;biomaterials;1
field;biomaterials;medical;1
biomaterials;medical;devices;1
medical;devices;biomaterials;1
devices;biomaterials;material;1
biomaterials;material;science;1
material;science;medical;1
science;medical;devices;1
medical;devices;tissue;1
devic

In [10]:
cmd = ("select t.text, cl.codclassificacao "
       "from resultadoanaliseprojetotweet r "
       "inner join classificacao cl on cl.codclassificacao = r.codclassificacao "
       "inner join tweet t on r.codtweet = t.codtweet "
       "where codresultadoanaliseprojeto = 20 and cl.codclassificacao = 228 ")
res01 = query(cmd)

In [11]:
print(len(res01))

9379


In [12]:
res01[:10]

[('Check out Dental Implant Screw Remover repair kit #Totalimplant https://t.co/5IEJBC0Umx via @eBay',
  228),
 ('Check out Dental Implant Screw Remover &amp; Repair Kit #Totalimplant https://t.co/uyq2fRXVej via @eBay',
  228),
 ('Check out Dental Implant Screw Remover broken screw kit #Totalimplant https://t.co/OBMoXwAiCD via @eBay',
  228),
 ('Cheap Dental Implants: 5 Tips to Find the Cheap Implants 8\n#Cheap #dentalimplants #implants\nhttps://t.co/2jmCppb7K7',
  228),
 ('Cheap Dental Implants: 5 Tips to Find the Cheap Implants 8\n#Cheap #dentalimplants #implants\nhttps://t.co/CjpeoyAUU6',
  228),
 ('Cheap Dental Implants: 5 Tips to Find the Cheap Implants 8\n#Cheap #dentalimplants #implants\nhttps://t.co/87U2jUoRag',
  228),
 ('Cheap Dental Implants: 5 Tips to Find the Cheap Implants 8\n#Cheap #dentalimplants #implants\nhttps://t.co/hXm07WBFRY',
  228),
 ('Check out Dental Implant thread repair kit #Totalimplant https://t.co/h0DdOEt4Ys via @eBay',
  228),
 ('Check out Dental Implant

In [13]:
txt = ''
for r in res01:
    txt = txt + ' ' + r[0].lower()

In [14]:
print(txt[:1000])

 check out dental implant screw remover repair kit #totalimplant https://t.co/5iejbc0umx via @ebay check out dental implant screw remover &amp; repair kit #totalimplant https://t.co/uyq2frxvej via @ebay check out dental implant screw remover broken screw kit #totalimplant https://t.co/obmoxwaicd via @ebay cheap dental implants: 5 tips to find the cheap implants 8
#cheap #dentalimplants #implants
https://t.co/2jmcppb7k7 cheap dental implants: 5 tips to find the cheap implants 8
#cheap #dentalimplants #implants
https://t.co/cjpeoyauu6 cheap dental implants: 5 tips to find the cheap implants 8
#cheap #dentalimplants #implants
https://t.co/87u2juorag cheap dental implants: 5 tips to find the cheap implants 8
#cheap #dentalimplants #implants
https://t.co/hxm07wbfry check out dental implant thread repair kit #totalimplant https://t.co/h0ddoet4ys via @ebay check out dental implant thread repair set #totalimplant https://t.co/xdgqmavant via @ebay check out dental implant thread repair surgical

In [15]:
aaaa = 'uga'
print(aaaa[1:])

ga


In [16]:
tokens = tknzr.tokenize(txt)
print(tokens[:10])
stop = set(stopwords.words('english')).union({'dental', 'implant', 'implants', 'rt'})
clean_tokens = []
for token in tokens:
    if token[:1] == '#':
        token = token[1:]
    if not token.isalpha():
        continue
    if token in stop:
        continue
    if token[:4] == 'http':
        continue
    if token[:1] == '@':
        continue
    if len(token) <= 3:
        continue
        
    clean_tokens.append(token)
print(clean_tokens[:10])

['check', 'out', 'dental', 'implant', 'screw', 'remover', 'repair', 'kit', '#totalimplant', 'https://t.co/5iejbc0umx']
['check', 'screw', 'remover', 'repair', 'totalimplant', 'check', 'screw', 'remover', 'repair', 'totalimplant']


In [17]:
ungs = ngrams(clean_tokens,1)
aaa = Counter(ungs).most_common()
for b in aaa:
    print(b)

(('insurance',), 1865)
(('handpiece',), 1347)
(('cost',), 1052)
(('abutment',), 996)
(('check',), 974)
(('surgical',), 967)
(('angle',), 937)
(('contra',), 927)
(('idea',), 887)
(('surprise',), 882)
(('sponsored',), 868)
(('listings',), 849)
(('surgery',), 823)
(('reduction',), 733)
(('speed',), 617)
(('motor',), 576)
(('screw',), 529)
(('save',), 462)
(('bone',), 448)
(('system',), 438)
(('money',), 435)
(('costa',), 426)
(('cheap',), 425)
(('rica',), 425)
(('restoration',), 418)
(('straight',), 402)
(('analog',), 374)
(('affordable',), 366)
(('tools',), 352)
(('brushless',), 351)
(('taking',), 345)
(('getting',), 344)
(('consider',), 341)
(('importance',), 338)
(('plunge',), 337)
(('best',), 336)
(('dentistry',), 332)
(('broken',), 298)
(('free',), 289)
(('instruments',), 252)
(('jersey',), 223)
(('external',), 219)
(('crown',), 211)
(('push',), 211)
(('allendaledentalimplants',), 201)
(('fortleedentalimplants',), 201)
(('remover',), 200)
(('garfielddenta',), 200)
(('removal',), 198)

(('dispos',), 1)
(('arrival',), 1)
(('hygiene',), 1)
(('morse',), 1)
(('lakhs',), 1)
(('seized',), 1)
(('convex',), 1)
(('redmond',), 1)
(('vineland',), 1)
(('silent',), 1)
(('generators',), 1)
(('nigeria',), 1)
(('caturday',), 1)
(('watering',), 1)
(('grass',), 1)
(('mcgraw',), 1)
(('hill',), 1)
(('economics',), 1)
(('answers',), 1)
(('mysterio',), 1)
(('holiday',), 1)
(('broadwayfamilydentalpc',), 1)
(('dentalimplantspecial',), 1)
(('steamboat',), 1)
(('springs',), 1)
(('champagne',), 1)
(('suject',), 1)
(('load',), 1)
(('balancing',), 1)
(('mattress',), 1)
(('cartersville',), 1)
(('core',), 1)
(('trophy',), 1)
(('melbourne',), 1)
(('dresses',), 1)
(('indonesian',), 1)
(('brochures',), 1)
(('korean',), 1)
(('panties',), 1)
(('beck',), 1)
(('verlag',), 1)
(('coin',), 1)
(('album',), 1)
(('pashmina',), 1)
(('shawls',), 1)
(('kuwait',), 1)
(('decor',), 1)
(('mustafa',), 1)
(('skull',), 1)
(('wholesale',), 1)
(('dhabi',), 1)
(('twitter',), 1)
(('socialbuttons',), 1)
(('organza',), 1)
(('

In [18]:
bgs = ngrams(clean_tokens,2)
aaa = Counter(bgs).most_common()
for b in aaa:
    print(b)

(('contra', 'angle'), 923)
(('cost', 'surprise'), 879)
(('idea', 'cost'), 849)
(('surprise', 'sponsored'), 849)
(('sponsored', 'listings'), 849)
(('listings', 'idea'), 543)
(('angle', 'handpiece'), 534)
(('costa', 'rica'), 425)
(('save', 'money'), 425)
(('money', 'costa'), 414)
(('rica', 'save'), 389)
(('speed', 'contra'), 386)
(('reduction', 'speed'), 381)
(('importance', 'insurance'), 338)
(('insurance', 'consider'), 338)
(('consider', 'taking'), 338)
(('taking', 'plunge'), 337)
(('plunge', 'getting'), 337)
(('surgery', 'tools'), 313)
(('abutment', 'analog'), 307)
(('dentistry', 'restoration'), 305)
(('restoration', 'surgery'), 305)
(('getting', 'importance'), 290)
(('broken', 'screw'), 254)
(('surgical', 'brushless'), 233)
(('brushless', 'motor'), 212)
(('reduction', 'contra'), 205)
(('best', 'jersey'), 201)
(('jersey', 'allendaledentalimplants'), 201)
(('allendaledentalimplants', 'fortleedentalimplants'), 201)
(('fortleedentalimplants', 'garfielddenta'), 200)
(('check', 'bone'), 18

(('plans', 'online'), 3)
(('bridge', 'photos'), 3)
(('photos', 'kerala'), 3)
(('kerala', 'online'), 3)
(('shopping', 'textiles'), 3)
(('sarees', 'online'), 3)
(('shopping', 'ohio'), 3)
(('cicada', 'insurance'), 3)
(('memphis', 'online'), 3)
(('shopping', 'coxo'), 3)
(('home', 'health'), 3)
(('instruments', 'pakistan'), 3)
(('tapered', 'screw'), 3)
(('screw', 'vent'), 3)
(('handpiece', 'costs'), 3)
(('india', 'reduction'), 3)
(('shopping', 'sites'), 3)
(('home', 'average'), 3)
(('home', 'sale'), 3)
(('rica', 'surgery'), 3)
(('screen', 'save'), 3)
(('super', 'save'), 3)
(('rica', 'check'), 3)
(('plans', 'late'), 3)
(('late', 'onset'), 3)
(('onset', 'multiple'), 3)
(('multiple', 'sclerosis'), 3)
(('care', 'glen'), 3)
(('glen', 'patient'), 3)
(('crown', 'popular'), 3)
(('screw', 'inte'), 3)
(('inte', 'sydent'), 3)
(('life', 'insurance'), 3)
(('individuals', 'insurance'), 3)
(('funeral', 'home'), 3)
(('coverage', 'free'), 3)
(('straight', 'shoulder'), 3)
(('cost', 'india'), 3)
(('surgery', 

(('deal', 'cheap'), 1)
(('biomet', 'totalimplantcom'), 1)
(('zimmer', 'analysis'), 1)
(('teeth', 'teaching'), 1)
(('teaching', 'model'), 1)
(('model', 'importance'), 1)
(('dentistry', 'sales'), 1)
(('london', 'croydon'), 1)
(('croydon', 'helix'), 1)
(('recruitment', 'croydon'), 1)
(('croydon', 'jobs'), 1)
(('jobs', 'lovejobs'), 1)
(('lovejobs', 'importance'), 1)
(('westbritonjobs', 'importance'), 1)
(('mixed', 'types'), 1)
(('mixed', 'euroteknika'), 1)
(('medwayjobs', 'importance'), 1)
(('straumann', 'anthogyr'), 1)
(('anthogyr', 'alphabio'), 1)
(('getting', 'screw'), 1)
(('cortex', 'totalimplant'), 1)
(('totalimplant', 'screw'), 1)
(('getting', 'analog'), 1)
(('analog', 'ball'), 1)
(('ship', 'speed'), 1)
(('speed', 'drivers'), 1)
(('drivers', 'surgical'), 1)
(('biomet', 'screw'), 1)
(('totalimplantcom', 'importance'), 1)
(('taking', 'importance'), 1)
(('solution', 'totalimplant'), 1)
(('astra', 'importance'), 1)
(('getting', 'drivers'), 1)
(('shipping', 'importance'), 1)
(('mozo', 'im

(('certified', 'used'), 1)
(('used', 'mini'), 1)
(('mini', 'save'), 1)
(('removal', 'save'), 1)
(('screw', 'save'), 1)
(('megagen', 'save'), 1)
(('plans', 'vintage'), 1)
(('vintage', 'shop'), 1)
(('online', 'ireland'), 1)
(('ireland', 'save'), 1)
(('sterile', 'save'), 1)
(('plans', 'fizz'), 1)
(('fizz', 'mode'), 1)
(('mode', 'online'), 1)
(('shop', 'save'), 1)
(('individuals', 'campomaggi'), 1)
(('campomaggi', 'bags'), 1)
(('bags', 'shop'), 1)
(('online', 'bone'), 1)
(('sterile', 'london'), 1)
(('london', 'tanmani'), 1)
(('tanmani', 'jewellery'), 1)
(('jewellery', 'online'), 1)
(('clothes', 'cheap'), 1)
(('cheap', 'malaysia'), 1)
(('malaysia', 'nobel'), 1)
(('cheap', 'dress'), 1)
(('dress', 'materials'), 1)
(('materials', 'online'), 1)
(('plans', 'china'), 1)
(('china', 'smartphones'), 1)
(('smartphones', 'online'), 1)
(('india', 'pieces'), 1)
(('costs', 'shop'), 1)
(('insurance', 'online'), 1)
(('online', 'reduction'), 1)
(('certain', 'compendial'), 1)
(('compendial', 'testing'), 1)
(

(('insurance', 'columbus'), 1)
(('columbus', 'ohio'), 1)
(('ohio', 'miami'), 1)
(('miami', 'beach'), 1)
(('beach', 'manufacturing'), 1)
(('manufacturing', 'ring'), 1)
(('ring', 'cent'), 1)
(('cent', 'price'), 1)
(('tooth', 'cargo'), 1)
(('cargo', 'shipping'), 1)
(('shipping', 'health'), 1)
(('cover', 'components'), 1)
(('crown', 'springfield'), 1)
(('cover', 'primerica'), 1)
(('primerica', 'jersey'), 1)
(('jersey', 'surgical'), 1)
(('guide', 'draft'), 1)
(('draft', 'forms'), 1)
(('forms', 'insurance'), 1)
(('cost', 'italytravel'), 1)
(('italytravel', 'insurance'), 1)
(('sandent', 'portland'), 1)
(('oregon', 'attorneys'), 1)
(('attorneys', 'idaho'), 1)
(('idaho', 'falls'), 1)
(('falls', 'idaho'), 1)
(('idaho', 'builders'), 1)
(('builders', 'risk'), 1)
(('risk', 'insurance'), 1)
(('quote', 'loose'), 1)
(('loose', 'houses'), 1)
(('houses', 'rent'), 1)
(('rent', 'carlisle'), 1)
(('carlisle', 'abutment'), 1)
(('abutment', 'startlogic'), 1)
(('startlogic', 'email'), 1)
(('email', 'tissue'), 

(('account', 'dentistry'), 1)
(('analog', 'smile'), 1)
(('info', 'dentistry'), 1)
(('tools', 'working'), 1)
(('home', 'works'), 1)
(('works', 'small'), 1)
(('small', 'jobs'), 1)
(('jobs', 'economy'), 1)
(('economy', 'judge'), 1)
(('judge', 'trial'), 1)
(('trial', 'assemble'), 1)
(('assemble', 'home'), 1)
(('home', 'covered'), 1)
(('finance', 'call'), 1)
(('affordable', 'sydney'), 1)
(('sydney', 'tooth'), 1)
(('sydney', 'sidejobsnearme'), 1)
(('sidejobsnearme', 'portland'), 1)
(('portland', 'ankylos'), 1)
(('ankylos', 'characteristics'), 1)
(('insurance', 'characteristics'), 1)
(('analog', 'coxo'), 1)
(('handpiece', 'portland'), 1)
(('bego', 'bone'), 1)
(('sterile', 'mini'), 1)
(('analog', 'portland'), 1)
(('tools', 'mini'), 1)
(('portland', 'system'), 1)
(('oregon', 'dentistry'), 1)
(('tools', 'specialist'), 1)
(('analog', 'miss'), 1)
(('miss', 'save'), 1)
(('flapless', 'invest'), 1)
(('invest', 'teeth'), 1)
(('teeth', 'month'), 1)
(('month', 'book'), 1)
(('consultation', 'today'), 1)


(('dentalinstruments', 'offer'), 1)
(('pasadena', 'offer'), 1)
(('listings', 'forget'), 1)
(('forget', 'cost'), 1)
(('cost', 'even'), 1)
(('even', 'piece'), 1)
(('piece', 'come'), 1)
(('come', 'done'), 1)
(('done', 'single'), 1)
(('tooth', 'surgery'), 1)
(('surgery', 'tinanium'), 1)
(('tinanium', 'abutment'), 1)
(('included', 'also'), 1)
(('also', 'find'), 1)
(('find', 'idea'), 1)
(('affordable', 'covered'), 1)
(('insurance', 'sadly'), 1)
(('sadly', 'neither'), 1)
(('neither', 'case'), 1)
(('case', 'idea'), 1)
(('surgical', 'workshop'), 1)
(('workshop', 'idea'), 1)
(('kits', 'idea'), 1)
(('listings', 'home'), 1)
(('home', 'kind'), 1)
(('kind', 'wealth'), 1)
(('wealth', 'status'), 1)
(('status', 'symbols'), 1)
(('symbols', 'guess'), 1)
(('check', 'vista'), 1)
(('vista', 'tunneling'), 1)
(('tunneling', 'procedure'), 1)
(('procedure', 'sinus'), 1)
(('lift', 'sterilize'), 1)
(('cassette', 'idea'), 1)
(('ratchet', 'wrench'), 1)
(('wrench', 'surgical'), 1)
(('surgical', 'guided'), 1)
(('guid

(('time', 'read'), 1)
(('prosthetics', 'soooo'), 1)
(('soooo', 'like'), 1)
(('like', 'grand'), 1)
(('grand', 'outrageous'), 1)
(('outrageous', 'insurance'), 1)
(('insurance', 'right'), 1)
(('right', 'read'), 1)
(('prosthetics', 'online'), 1)
(('online', 'academy'), 1)
(('academy', 'alert'), 1)
(('alert', 'uploaded'), 1)
(('uploaded', 'lecture'), 1)
(('lecture', 'recording'), 1)
(('recording', 'watch'), 1)
(('watch', 'preview'), 1)
(('preview', 'outcomes'), 1)
(('outcomes', 'immediate'), 1)
(('immediate', 'early'), 1)
(('early', 'loading'), 1)
(('loading', 'year'), 1)
(('year', 'vladimir'), 1)
(('vladimir', 'kokovic'), 1)
(('kokovic', 'ebook'), 1)
(('prosthetics', 'drivers'), 1)
(('knob', 'broken'), 1)
(('zahnimplantate', 'fractured'), 1)
(('lineal', 'epub'), 1)
(('treatment', 'epub'), 1)
(('prosthetics', 'dentistry'), 1)
(('dentistry', 'practices'), 1)
(('practices', 'crown'), 1)
(('tooth', 'article'), 1)
(('article', 'page'), 1)
(('page', 'journal'), 1)
(('journal', 'info'), 1)
(('inf

In [19]:
tgs = ngrams(clean_tokens,3)
aaa = Counter(tgs).most_common()
for b in aaa:
    print(b)

(('idea', 'cost', 'surprise'), 849)
(('cost', 'surprise', 'sponsored'), 849)
(('surprise', 'sponsored', 'listings'), 849)
(('sponsored', 'listings', 'idea'), 543)
(('listings', 'idea', 'cost'), 543)
(('contra', 'angle', 'handpiece'), 524)
(('save', 'money', 'costa'), 414)
(('money', 'costa', 'rica'), 414)
(('costa', 'rica', 'save'), 389)
(('rica', 'save', 'money'), 389)
(('speed', 'contra', 'angle'), 386)
(('importance', 'insurance', 'consider'), 338)
(('insurance', 'consider', 'taking'), 338)
(('consider', 'taking', 'plunge'), 337)
(('taking', 'plunge', 'getting'), 337)
(('reduction', 'speed', 'contra'), 321)
(('dentistry', 'restoration', 'surgery'), 305)
(('restoration', 'surgery', 'tools'), 305)
(('plunge', 'getting', 'importance'), 290)
(('getting', 'importance', 'insurance'), 290)
(('reduction', 'contra', 'angle'), 204)
(('best', 'jersey', 'allendaledentalimplants'), 201)
(('jersey', 'allendaledentalimplants', 'fortleedentalimplants'), 201)
(('allendaledentalimplants', 'fortleeden

(('transfer', 'open', 'tray'), 4)
(('open', 'tray', 'abutment'), 4)
(('abutment', 'impression', 'coping'), 4)
(('school', 'save', 'money'), 4)
(('complete', 'handpiece', 'surgical'), 4)
(('angle', 'handpiece', 'bone'), 4)
(('optic', 'handpiece', 'reduction'), 4)
(('insurance', 'coverage', 'reduction'), 4)
(('pudsey', 'dentist', 'finance'), 4)
(('dentist', 'finance', 'available'), 4)
(('osteotomes', 'straight', 'concave'), 4)
(('handpiece', 'slow', 'speed'), 4)
(('reduction', 'push', 'contra'), 4)
(('handpiece', 'speed', 'reduction'), 4)
(('smax', 'reduction', 'speed'), 4)
(('handpiece', 'benefits', 'produced'), 4)
(('handpiece', 'mini', 'type'), 4)
(('handpiece', 'push', 'spray'), 4)
(('abutment', 'shoulder', 'slim'), 4)
(('shoulder', 'slim', 'platform'), 4)
(('surgical', 'speed', 'contra'), 4)
(('angle', 'push', 'reduction'), 4)
(('push', 'reduction', 'handpiece'), 4)
(('speed', 'handpiece', 'bone'), 4)
(('sinus', 'lift', 'orthodontic'), 4)
(('lift', 'orthodontic', 'oral'), 4)
(('orth

(('zahnimplantate', 'totalimplantcom', 'check'), 3)
(('read', 'online', 'science'), 3)
(('kindle', 'free', 'prosthetics'), 3)
(('zahnimplantate', 'idea', 'cost'), 3)
(('check', 'ruler', 'locator'), 3)
(('ruler', 'locator', 'size'), 3)
(('locator', 'size', 'estimates'), 3)
(('size', 'estimates', 'dentaires'), 3)
(('science', 'epub', 'free'), 3)
(('standard', 'internal', 'ball'), 3)
(('caps', 'internal', 'ball'), 3)
(('mobi', 'prosthetics', 'read'), 3)
(('prosthetics', 'read', 'free'), 3)
(('free', 'prosthetics', 'mobi'), 3)
(('prosthetics', 'mobi', 'free'), 3)
(('download', 'free', 'complications'), 3)
(('check', 'tissue', 'punches'), 3)
(('free', 'science', 'kindle'), 3)
(('listings', 'read', 'mobi'), 3)
(('mobi', 'prosthetics', 'download'), 3)
(('prosthetics', 'read', 'prosthetics'), 3)
(('prosthetics', 'download', 'kindle'), 3)
(('september', 'awareness', 'month'), 3)
(('awareness', 'month', 'sponsored'), 3)
(('month', 'sponsored', 'american'), 3)
(('sponsored', 'american', 'academy'

(('cavity', 'balloon', 'pricey'), 2)
(('balloon', 'pricey', 'extensive'), 2)
(('pricey', 'extensive', 'problem'), 2)
(('extensive', 'problem', 'requires'), 2)
(('problem', 'requires', 'crown'), 2)
(('requires', 'crown', 'root'), 2)
(('crown', 'root', 'canal'), 2)
(('root', 'canal', 'insurance'), 2)
(('canal', 'insurance', 'helps'), 2)
(('insurance', 'helps', 'timely'), 2)
(('helps', 'timely', 'care'), 2)
(('timely', 'care', 'teeth'), 2)
(('care', 'teeth', 'implantsprocenter'), 2)
(('teeth', 'implantsprocenter', 'dentalimplants'), 2)
(('implantsprocenter', 'dentalimplants', 'insurance'), 2)
(('listings', 'bone', 'compression'), 2)
(('screws', 'sinus', 'lift'), 2)
(('sinus', 'lift', 'expander'), 2)
(('lift', 'expander', 'surgical'), 2)
(('expander', 'surgical', 'tool'), 2)
(('surgical', 'tool', 'offer'), 2)
(('tool', 'offer', 'complete'), 2)
(('drill', 'stopper', 'surgical'), 2)
(('stopper', 'surgical', 'drill'), 2)
(('orthopedic', 'instruments', 'plastic'), 2)
(('email', 'subject', 'lin

(('abutment', 'ridge', 'split'), 1)
(('split', 'compressor', 'short'), 1)
(('compressor', 'short', 'impression'), 1)
(('short', 'impression', 'coping'), 1)
(('tray', 'abutment', 'ebay'), 1)
(('abutment', 'ebay', 'deals'), 1)
(('tooth', 'idea', 'anti'), 1)
(('idea', 'anti', 'rotational'), 1)
(('nobel', 'replace', 'transfer'), 1)
(('replace', 'transfer', 'impression'), 1)
(('tray', 'abutment', 'fortis'), 1)
(('abutment', 'fortis', 'straight'), 1)
(('healing', 'internal', 'chelsea'), 1)
(('internal', 'chelsea', 'available'), 1)
(('chelsea', 'available', 'month'), 1)
(('available', 'month', 'interface'), 1)
(('month', 'interface', 'abutment'), 1)
(('package', 'deal', 'analysis'), 1)
(('deal', 'analysis', 'crown'), 1)
(('model', 'sale', 'cheap'), 1)
(('sale', 'cheap', 'tips'), 1)
(('cheap', 'dentalimplants', 'smile'), 1)
(('dentalimplants', 'smile', 'makeover'), 1)
(('makeover', 'enter', 'machine'), 1)
(('enter', 'machine', 'drill'), 1)
(('drill', 'motor', 'system'), 1)
(('motor', 'system',

...

(('analog', 'root', 'canal'), 1)
(('save', 'tooth', 'huff'), 1)
(('tooth', 'huff', 'dover'), 1)
(('huff', 'dover', 'discusses'), 1)
(('dover', 'discusses', 'dentistry'), 1)
(('discusses', 'dentistry', 'restoration'), 1)
(('zimmer', 'straumann', 'abutment'), 1)
(('straumann', 'abutment', 'analog'), 1)
(('tools', 'surgery', 'irrigation'), 1)
(('surgical', 'handpiece', 'speed'), 1)
(('button', 'chuck', 'dentistry'), 1)
(('chuck', 'dentistry', 'restoration'), 1)
(('check', 'surgery', 'advance'), 1)
(('surgery', 'advance', 'sinus'), 1)
(('advance', 'sinus', 'lift'), 1)
(('lift', 'instruments', 'icymi'), 1)
(('instruments', 'icymi', 'root'), 1)
(('icymi', 'root', 'canal'), 1)
(('save', 'tooth', 'discuss'), 1)
(('tooth', 'discuss', 'abutment'), 1)
(('discuss', 'abutment', 'analog'), 1)
(('tools', 'tooth', 'filling'), 1)
(('tooth', 'filling', 'crack'), 1)
(('filling', 'crack', 'need'), 1)
(('crack', 'need', 'questions'), 1)
(('need', 'questions', 'insurance'), 1)
(('questions', 'insurance', 'i

(('surgery', 'platelet', 'rich'), 1)
(('rich', 'fibrin', 'robot'), 1)
(('fibrin', 'robot', 'successfully'), 1)
(('human', 'futurist', 'straight'), 1)
(('futurist', 'straight', 'anatomic'), 1)
(('connection', 'nobel', 'usps'), 1)
(('nobel', 'usps', 'system'), 1)
(('azdent', 'handpiece', 'surgery'), 1)
(('endo', 'system', 'robot'), 1)
(('system', 'robot', 'successfully'), 1)
(('external', 'nozzle', 'yusen'), 1)
(('nozzle', 'yusen', 'handpiece'), 1)
(('yusen', 'handpiece', 'contra'), 1)
(('angle', 'speed', 'robot'), 1)
(('speed', 'robot', 'successfully'), 1)
(('human', 'futurist', 'anatomic'), 1)
(('futurist', 'anatomic', 'straight'), 1)
(('connection', 'nobel', 'hongkong'), 1)
(('nobel', 'hongkong', 'jobs'), 1)
(('please', 'page', 'hongkong'), 1)
(('page', 'hongkong', 'jobs'), 1)
(('hong', 'kong', 'contract'), 1)
(('kong', 'contract', 'precious'), 1)
(('contract', 'precious', 'smile'), 1)
(('please', 'page', 'anatomic'), 1)
(('page', 'anatomic', 'straight'), 1)
(('connection', 'nobel', '

(('internal', 'abutment', 'soft'), 1)
(('abutment', 'soft', 'silicon'), 1)
(('internal', 'abutment', 'affordable'), 1)
(('abutment', 'affordable', 'health'), 1)
(('insurance', 'insurance', 'visit'), 1)
(('insurance', 'visit', 'idea'), 1)
(('cost', 'surprise', 'want'), 1)
(('surprise', 'want', 'white'), 1)
(('want', 'white', 'teeth'), 1)
(('white', 'teeth', 'whiten'), 1)
(('teeth', 'whiten', 'brighten'), 1)
(('whiten', 'brighten', 'coupons'), 1)
(('brighten', 'coupons', 'please'), 1)
(('sponsored', 'listings', 'announced'), 1)
(('listings', 'announced', 'unbelievable'), 1)
(('announced', 'unbelievable', 'grand'), 1)
(('unbelievable', 'grand', 'opening'), 1)
(('grand', 'opening', 'sale'), 1)
(('opening', 'sale', 'bestdentalmplants'), 1)
(('sale', 'bestdentalmplants', 'placed'), 1)
(('bestdentalmplants', 'placed', 'bestperiodontist'), 1)
(('placed', 'bestperiodontist', 'flat'), 1)
(('bestperiodontist', 'flat', 'rate'), 1)
(('flat', 'rate', 'maint'), 1)
(('rate', 'maint', 'incl'), 1)
(('ma

(('taste', 'mouth', 'along'), 1)
(('mouth', 'along', 'lines'), 1)
(('along', 'lines', 'check'), 1)
(('lines', 'check', 'specialist'), 1)
(('check', 'specialist', 'guess'), 1)
(('specialist', 'guess', 'much'), 1)
(('brushless', 'motor', 'brand'), 1)
(('motor', 'brand', 'ebook'), 1)
(('brand', 'ebook', 'available'), 1)
(('ebook', 'available', 'current'), 1)
(('available', 'current', 'trends'), 1)
(('current', 'trends', 'beginnings'), 1)
(('trends', 'beginnings', 'retaining'), 1)
(('beginnings', 'retaining', 'restorations'), 1)
(('retaining', 'restorations', 'free'), 1)
(('restorations', 'free', 'download'), 1)
(('free', 'download', 'commercial'), 1)
(('download', 'commercial', 'supporter'), 1)
(('commercial', 'supporter', 'guess'), 1)
(('supporter', 'guess', 'much'), 1)
(('cost', 'surprise', 'coxo'), 1)
(('surprise', 'coxo', 'redution'), 1)
(('button', 'handpiece', 'something'), 1)
(('handpiece', 'something', 'comes'), 1)
(('something', 'comes', 'something'), 1)
(('comes', 'something', '

(('titanium', 'internal', 'surgical'), 1)
(('internal', 'surgical', 'prep'), 1)
(('diffusedentalbomb', 'newimplantresearc', 'contra'), 1)
(('newimplantresearc', 'contra', 'angle'), 1)
(('handpiece', 'surgical', 'prep'), 1)
(('diffusedentalbomb', 'newimplantresearc', 'download'), 1)
(('newimplantresearc', 'download', 'free'), 1)
(('free', 'prosthetics', 'surgical'), 1)
(('diffusedentalbomb', 'newimplantresearc', 'total'), 1)
(('newimplantresearc', 'total', 'package'), 1)
(('analog', 'abutment', 'please'), 1)
(('bioconcept', 'details', 'portland'), 1)
(('details', 'portland', 'sales'), 1)
(('portland', 'sales', 'consultant'), 1)
(('working', 'clearchoice', 'download'), 1)
(('clearchoice', 'download', 'prosthetics'), 1)
(('download', 'prosthetics', 'fiber'), 1)
(('prosthetics', 'fiber', 'optic'), 1)
(('contra', 'angle', 'idea'), 1)
(('conical', 'connection', 'angled'), 1)
(('connection', 'angled', 'multi'), 1)
(('conical', 'connection', 'access'), 1)
(('connection', 'access', 'prosthetics